# Laboratorio 5: Deep learning (Perros y gatos)

Procedimiento:

1.   Cargar la base de datos ([https://www.kaggle.com/chetankv/dogs-cats-images?](https://www.kaggle.com/chetankv/dogs-cats-images?))
2.   Realizar las transformaciones



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Cargar las librerías

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

## 2. Cargar imágenes

Se cargan las imágenes utilizando la clase ImageFolder. La clase ImageForder etiqueta automáticamente las imágenes dada una ruta donde se encuentran los datos. Por ejemplo, si tengo el directorio raíz dataset/training_set. Debo dejar mis imágenes de cada clase en una carpeta diferente. Si tengo imágenes de gatos debo dejarlos en dataset/training_set/cats, si tengo imágenes de perros las debo dejar en dataset/training_set/dogs, etc.

In [ ]:
import PIL

data_transform = transforms.Compose([
    transforms.Resize((128,128)),  
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomVerticalFlip(p=0.6),
    transforms.RandomHorizontalFlip(p=0.6),                              
    transforms.RandomChoice([
      transforms.RandomAffine(degrees=0, translate=(0.1, 0.1),
                          scale=None, shear=None,
                          resample=False, fillcolor=0),
      transforms.ColorJitter(brightness=(1, 1.5),
                            contrast=(0.3, 2),
                            saturation=(0.1, 1),
                            hue=(-0.3, 0.3)),
      transforms.RandomRotation((-10, 10), resample=PIL.Image.BILINEAR)
    ]),
    transforms.ToTensor()
])

gatos_perros_dataset = datasets.ImageFolder(root='/content/drive/My Drive/D-UCN/Classes/TecnicasAvanzadasAprendizajeAutomatico/Laboratorios/Laboratorio05.2:DeepLearning/dataset/training_set',
                                            transform=data_transform)

print(gatos_perros_dataset)

dataset_loader = torch.utils.data.DataLoader(gatos_perros_dataset,
                                             batch_size=32,
                                             shuffle=False,
                                             num_workers=2)

print(dataset_loader)

## 3. Visualizo las imágenes cargadas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(num=None, figsize=(12, 8), dpi=80)

def imshow(img):
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img,(1, 2, 0)))

# Obtener imagenes
data_iter = iter(dataset_loader)
images, labels = data_iter.next()

# Mostrar imagenes
imshow(torchvision.utils.make_grid(images))

## 4. Separando la base de datos

Para realizar el entrenamiento y pruebas se recomienda separar los datos en tres conjuntos:

* **Conjunto de entrenamiento:** El modelo aprende de los ejemplos de este conjunto de datos. Se ajusta un parámetro a un clasificador.
* **Conjunto de validación:** Los ejemplos en el conjunto de datos de validación se utilizan para ajustar los hiperparámetros, como la tasa de aprendizaje y las épocas. El objetivo de crear un conjunto de validación es evitar un sobreajuste grande del modelo. Es un punto de control para saber si el modelo se ajusta bien con el conjunto de datos de entrenamiento.
* **Conjunto de pruebas:** Este conjunto de datos prueba la evolución final del modelo, midiendo qué tan bien aprendió y predijo el resultado deseado. Contiene datos invisibles de la vida real. Es un conjunto diferente al de entrenamiento y validación.

Imagen: [https://i.imgur.com/DV80uhS.png](https://i.imgur.com/DV80uhS.png)

In [ ]:
# Transformaciones de la imagen
import PIL

#data_transform = transforms.Compose([
#    transforms.Resize((32,32)),  
#    transforms.RandomGrayscale(p=0.2),
#    transforms.RandomVerticalFlip(p=0.2),
#    transforms.RandomHorizontalFlip(p=0.2),                              
#    transforms.ToTensor()
#])

data_transform = transforms.Compose([
    transforms.Resize((32,32)),  
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomVerticalFlip(p=0.6),
    transforms.RandomHorizontalFlip(p=0.6),                              
    transforms.RandomChoice([
      transforms.RandomAffine(degrees=0, translate=(0.1, 0.1),
                          scale=None, shear=None,
                          resample=False, fillcolor=0),
      transforms.ColorJitter(brightness=(1, 1.5),
                            contrast=(0.3, 2),
                            saturation=(0.1, 1),
                            hue=(-0.3, 0.3)),
      transforms.RandomRotation((-10, 10), resample=PIL.Image.BILINEAR)
    ]),
    transforms.ToTensor()
])

data_transform_test = transforms.Compose([
    transforms.Resize((32,32)),                            
    transforms.ToTensor()
])

# Cargar las imágenes
gatos_perros_train = datasets.ImageFolder(root='/content/drive/My Drive/D-UCN/Classes/TecnicasAvanzadasAprendizajeAutomatico/Laboratorios/Laboratorio05.2:DeepLearning/dataset/training_set',
                                          transform=data_transform)
gatos_perros_valid = datasets.ImageFolder(root='/content/drive/My Drive/D-UCN/Classes/TecnicasAvanzadasAprendizajeAutomatico/Laboratorios/Laboratorio05.2:DeepLearning/dataset/valid_set',
                                          transform=data_transform_test)
gatos_perros_test = datasets.ImageFolder(root='/content/drive/My Drive/D-UCN/Classes/TecnicasAvanzadasAprendizajeAutomatico/Laboratorios/Laboratorio05.2:DeepLearning/dataset/test_set',
                                          transform=data_transform_test)

# Conjunto de entrenamiento
train_loader = torch.utils.data.DataLoader(gatos_perros_train,
                                           batch_size=32,
                                           shuffle=True,
                                           num_workers=2)

# Conjunto de validacion
valid_loader = torch.utils.data.DataLoader(gatos_perros_valid,
                                           batch_size=32,
                                           shuffle=False,
                                           num_workers=2)

# Conjunto de pruebas
test_loader = torch.utils.data.DataLoader(gatos_perros_test,
                                           batch_size=32,
                                           shuffle=False,
                                           num_workers=2)

### Revisamos las imagenes de las nuevas cargas



In [ ]:
plt.figure(num=None, figsize=(12, 8), dpi=80)

# Obtener imagenes
data_iter = iter(test_loader)
images, labels = data_iter.next()

# Mostrar imagenes
imshow(torchvision.utils.make_grid(images))

## 5. El modelo

Construimos un modelo con dos capas convolucionales, un dropout y dos capas completamente conectadas para la clasificación.

* Imagen red: [https://i.imgur.com/wiP9IwZ.png](https://i.imgur.com/wiP9IwZ.png)
* Imagen Conv: https://i.imgur.com/E9GGJMJ.gif
* Imagen Pool: https://i.imgur.com/kSwID7J.png

In [ ]:
import torch
import torch.nn as nn # Módulo están los modelos de red
import torch.nn.functional as F # Módulo donde están las funciones de activación

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(720, 1024)
    self.fc2 = nn.Linear(1024, 2)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return x


### Creamos el modelo

In [ ]:
model = CNN()
print(model)

## 6. Optimización

Seleccionamos cualquier algoritmo optimizador disponible en el paquete `torch.optim`. En general, los optimizadores son moficaciones del descenso del gradiente. Cambiando los parámetros del modelo, como los pesos, y añadiendo sesgo, el modelo puede ser optimizado. La **tasa de aprendizaje** decidirá cuán grandes deben ser los pasos para cambiar los parámetros.

* Calcular lo que un pequeño cambio en cada peso haría a la función de pérdida (seleccionando la dirección para alcanzar los mínimos).
* Ajustar cada peso en función de su gradiente (es decir, dar un pequeño paso en la dirección determinada).
* Continúe haciendo los pasos 1 y 2 hasta que la función de pérdida sea lo más baja posible.

Aquí, la estimación del momento adaptativo (*adaptive moment estimation* - Adam) se utiliza como un optimizador. Es una mezcla de **RMSprop** y **descenso de gradiente estocástico**.

### Configuración de Hyperparámetros

In [ ]:
num_epochs = 500
num_classes = 2
learning_rate = 0.001

### Configuración del dispositivo

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Optimizador

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 7. Entrenamiento

In [ ]:
# Guardar las perdidas
train_losses = []
valid_losses = []

for epoch in range(1, num_epochs+1):
  ## Calcular el loss por cada iteracion
  train_loss = 0.0
  valid_loss = 0.0
  # Entrenar el model
  model.train()
  for data, target in train_loader:

    # Mover a GPU
    data = data.to(device)
    target = target.to(device)

    # Limpiar gradientes
    optimizer.zero_grad()
    # Prediccion
    output = model(data)
    # Medir el error
    loss = criterion(output, target)
    # Calcular gradientes
    loss.backward()
    # Actualizamos los pesos (Adam)
    optimizer.step()
    # Historia de la perdida
    train_loss += loss.item() * data.size(0)
  
  # Validar modelo (No calcula graientes, no actualizo pesos, no dropout)
  model.eval()
  for data, target in valid_loader:
     # Mover a GPU
    data = data.to(device)
    target = target.to(device) 

    # Prediccion para el conjunto de validacion
    output = model(data)

    loss = criterion(output, target)  

    valid_loss += loss.item() * data.size(0)

  # Calculo el promedio de las perdidas
  train_loss = train_loss/len(train_loader.sampler)
  valid_loss = valid_loss/len(valid_loader.sampler)

  train_losses.append(train_loss)
  valid_losses.append(valid_loss)

  # Imprimimos las perdidas
  print('Epoch: {} \tPerdida de entrenamiento: {:.6f} \tPerdida de Validacion: {:.6f}'.format(epoch, train_loss, valid_loss))
  

## 8. Pruebas


In [ ]:
# test-the-model
model.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save 
torch.save(model.state_dict(), 'model.ckpt')

9. Graficamos los resultados

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)